# Technique - Recursion (General)

Recursion refers to self-referential algorithms. Any algorithm that can be solved iteratively can be solved recursively and vice versa (though sometimes the iterative implementation requires a stack, as recursion implicitly invokes the call stack). Some problems are easier to implement with one or the other; go with whatever feels most natural. 

When considering a recursive solution, you must identify:
- A recurrence relationship in the problem. It helps to write this out explicitly.
- A base case; this is a case where no self-reference is needed to provide a result.
- A recursive case; the cases where self reference is required. 

With this, you can then begin implementing a recursive function. Here's a simple template:
```
def recurse(value):
    if base case:
        return base value
    else:
        return recurse(next value) + recurse(other next value)

def handler(input):
    return recurse(first value)
```

If your language allows it, you can implement the recursive function inside the handler function. I find that this is helpful if the recursive function needs to access a global data structure, but it can complicate testing:
```
def handler(input):
    def recurse(value):
        if base case:
            return base value
        else:
            return recurse(next value) + recurse(other next value)
return recurse(first value)
```

Some common gotchas:
- Not identifying the recurrence relationship completely before implementing the recursive function will cause confusion and possibly lead you to miss edge cases. 
- Every recursive call must eventually hit the base case; otherwise you will recurse forever. 
- A recursive function usually should return a concrete value in the base case or a combination of return values for recursive cases. A common mistake is to forget to return a recursive call's return value.

## [Letter Combinations of a Phone Number](https://leetcode.com/problems/letter-combinations-of-a-phone-number/)

Recurrence: To get all strings for the `i`th number in the input string, combine all of the `i`th numbers digits with all strings associated with the `i-1`th number. 

Suppose we have the following functions:
- `get_chars(i)` - returns characters associated with digit, e.g. `get_chars(2) -> ['a','b','c']`
- `combine(c, strs)` - given a char `c` and list of strings `strs`, returns a list containing every string in `strs` with `c` appended.  
- `get_combinations(s, i)` - a function that solves the problem, i.e. returns all letter combinations for an int string `s` of length `i`.

Base case: A single number `i`; return `get_chars(i)`
Recursive case: the `i`th number of our input string. `get_combinations(s, i) = combine(get_combinations(s, i-1), c) for c in get_chars(s[i])`

I actually got this as an interview problem once (during a period of my life when I was incompetent at interviewing) and promptly failed it after attempting a solution on the whiteboard with multiple layers of nested loops. Despite the experience leaving me a scarred and broken man (most medical experts agree that whiteboards are a worldwide leading cause of early death), I somehow got an offer and the interviewer later became my manager.

In [14]:
PHONE_PAD = {
    '2': ["a","b", "c"],
    '3': ["d","e","f"],
    '4': ["g","h","i"],
    '5': ["j","k","l"],
    '6': ["m","n","o"],
    '7': ["p","q","r","s"],
    '8': ["t","u","v"],
    '9': ["w","x","y","z"],
}

def combine(c, strs):
    return [c+string for string in strs] if strs else [c]

def get_combinations(nums, i=0):
    if i >= len(nums):
        return []
    solution = []
    for c in PHONE_PAD[nums[i]]:
        solution += combine(c, get_combinations(nums, i+1))
    return solution

print(get_combinations("23"))
print(get_combinations(""))
print(get_combinations("2"))

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']
[]
['a', 'b', 'c']


## [Validate binary search tree](https://leetcode.com/problems/validate-binary-search-tree/)

For a leaf node `n` and internal node `m`, `m` is a right parent if `n` is in `m`'s right subtree, and a `left` parent if the opposite. In the tree below, `-2`'s `left ancestors` are `4`, `2`, and `0`, its only right ancestor is `-5`:
```
        4
       / \
      2   6
     /   / \
   -5   5   7
     \
      0
     /
   -2  
```
Recurrence: A node is valid if the BST property holds for it and its children, and if `max(right ancestors) < node.val < min(left ancestors)`.
Base case: A root node; check if the BST property holds for the children and the node.
Recursive case: Check the BST property, `max(right ancestors) < node.val < min(left ancestors)`

An alternative approach is to do an in-order traversal and confirm that the node order is sorted, which is also implemented below.

In [19]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def isValidBST(self, node: TreeNode, r_ancestor=None, l_ancestor=None) -> bool:
        return not node or \
               (not node.left or node.left.val < node.val) and \
               (not node.right or node.val < node.right.val) and \
               (not r_ancestor or r_ancestor < node.val) and \
               (not l_ancestor or node.val < l_ancestor) and \
               self.isValidBST(node.left, r_ancestor, node.val) and \
               self.isValidBST(node.right, node.val, l_ancestor)

class Solution:
    def isValidBST(self, node: TreeNode) -> bool:
        self.prev = -float('inf')
        def in_order(node):
            if not node: return True
            valid = in_order(node.left)
            valid &= self.prev  < node.val
            self.prev  = node.val
            valid &= in_order(node.right)
            return valid
        return in_order(node)
            

## [Split BST](https://leetcode.com/problems/split-bst/)

An `O(N)` solution is possible and poorly explained on Leetcode. For a given tree T, we return `left` (every node in T where node.val <= V) and `right` (every node in T where node.val > V). Our base case is a single node - it either winds up in a left or right tree of one node, and the other tree is empty. For more than one node, we look at `root`. If `root` goes into `left`, we don't need to look at `root.left` since every value in the root's left subtree is also less than V, so we go on to look at `root.right`. Opposite logic applies if `root` goes into `right`; we go on to look at `root.left`. 

Let's say `V= 57` and we have this tree:
```
    0
   / \ 
-100  100
 / \  / \
   ....
```
We don't need to examine `root.left` since every value in it is less than 0, so we recurse into `root.right`. We then have three trees: the original `root` at `0`, and the `left` and `right` on our recursive call `split(root.right)`. In this case, we could merge them by setting `root.right = left`; every value in `root`'s original right subtree was greater than `root`, so this preserves the BST property. Then the recursive call's right is `right` and the original root is `left`, and we return both. Again, opposite logic applies if we examine `root.left` in the first case. I won't bother stating the recurrence here since it's more confusing than it's worth.

In [61]:
class Solution:    
    def splitBST(self, root: TreeNode, V: int) -> List[TreeNode]:
        if not root: return [None, None]
        if V >= root.val:
            left = root
            sub_left, right = self.splitBST(root.right, V)
            left.right = sub_left
        else:
            right = root
            left, sub_right = self.splitBST(root.left, V)
            right.left = sub_right
        return [left, right]

An alternative, brute-force `O(nlogn)` solution is to pre-order traverse the input tree and insert every node into the new left or right subtree. 

In [55]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def insert(root, new_node):
    if not root:
        return new_node
    if new_node.val <= root.val:
        root.left = insert(root.left, new_node) 
    else:
        root.right = insert(root.right, new_node)
    return root
    
class Solution:    
    def splitBST(self, root: TreeNode, V: int) -> List[TreeNode]:
        self.ans = [None, None]

        def preorder(root, V):
            if not root: return
            new_node = TreeNode(root.val)
            if root.val <= V: 
                self.ans[0] = insert(self.ans[0], new_node)
            else:
                self.ans[1] = insert(self.ans[1], new_node)
            preorder(root.left, V)
            preorder(root.right, V)
            
        preorder(root, V)
        return self.ans

## [Merge Two Sorted Lists](https://leetcode.com/problems/merge-two-sorted-lists/)

Can use an iterative mergesort merge here easily. How can we do this recursively?

In [28]:
class ListNode:
    def __init__(self, val, next):
        self.val = val
        self.next = next

class Solution:
    def mergeTwoLists(self, l1: ListNode, l2: ListNode) -> ListNode:
        if not (l1 and l2):
            return l1 if not l2 else l2
        if l1.val <= l2.val: 
            next_node = l1
            l1 = l1.next
        else:
            next_node = l2
            l2 = l2.next
        next_node.next = self.mergeTwoLists(l1,l2)
        return next_node

## [Swap Nodes in Pairs](https://leetcode.com/problems/swap-nodes-in-pairs/)

For this one, our recurrence is: `swapped list of N nodes = 2nd node + 1st node + swapped list of N-2 nodes`, with base cases for zero and one node.

In [ ]:
class Solution:
    def swapPairs(self, head: ListNode) -> ListNode:
        if not head:
            return None
        second = head.next
        if not second:
            return head
        head.next = self.swapPairs(second.next)
        second.next = head
        return second

## [All Possible Full Binary Trees](https://leetcode.com/problems/all-possible-full-binary-trees/)